# Reducción de Overfitting - Técnicas Aplicables a SVM

## ⚠️ IMPORTANTE: Técnicas para SVM (NO para redes neuronales)

**Técnicas que NO aplican a SVM:**
- ❌ Dropout (solo para redes neuronales)
- ❌ Early Stopping (solo para redes neuronales)

**Técnicas que SÍ aplican a SVM:**
- ✅ **Class Weights** (balanceo de clases)
- ✅ **Regularización L2** (parámetro C en SVM)
- ✅ **Data Augmentation** (aumento de datos)
- ✅ **Cross-validation** (validación cruzada)
- ✅ **Reducir complejidad** (menos features, vectorizador más simple)

Este notebook implementa todas las técnicas aplicables a SVM.


In [96]:
# Librerías
import pandas as pd
import numpy as np
import pickle
import random

from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score, StratifiedKFold
import optuna

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

# Para sinónimos - NLTK (backup)
try:
    import nltk
    from nltk.corpus import wordnet as wn
    from nltk.tokenize import word_tokenize
    HAS_WORDNET = True
    # Descargar recursos si no están
    try:
        nltk.data.find('tokenizers/punkt')
    except LookupError:
        nltk.download('punkt', quiet=True)
    try:
        nltk.data.find('corpora/wordnet')
    except LookupError:
        nltk.download('wordnet', quiet=True)
except ImportError:
    HAS_WORDNET = False
    print("⚠️  NLTK no disponible.")

# Para data augmentation mejorada - nlpaug
try:
    import nlpaug.augmenter.word as naw
    import nlpaug.augmenter.sentence as nas
    HAS_NLPAUG = True
    print("✅ nlpaug disponible")
except ImportError:
    HAS_NLPAUG = False
    print("⚠️  nlpaug no instalado.")
    print("   Para instalar: pip install nlpaug")
    print("   El notebook funcionará con WordNet como backup")

np.random.seed(42)
random.seed(42)

print("✅ Librerías importadas")


⚠️  nlpaug no instalado.
   Para instalar: pip install nlpaug
   El notebook funcionará con WordNet como backup
✅ Librerías importadas


## ⚙️ Instalación Opcional de nlpaug

Si quieres usar nlpaug para mejor augmentación, ejecuta la siguiente celda.
Si no, el notebook funcionará con WordNet como backup.


In [97]:
# OPCIONAL: Instalar nlpaug para mejor augmentación
# Descomenta las siguientes líneas si quieres instalar nlpaug:
# import sys
# !{sys.executable} -m pip install nlpaug
# print("✅ nlpaug instalado. Reinicia el kernel y vuelve a ejecutar desde el inicio.")

# Por ahora, continuamos con WordNet como backup
print("ℹ️  Continuando con WordNet como backup. nlpaug se usará si está disponible.")


ℹ️  Continuando con WordNet como backup. nlpaug se usará si está disponible.


In [98]:
# Cargar datos
df = pd.read_csv('../data/processed/youtoxic_english_1000_processed.csv')
with open('../data/processed/y_train.pkl', 'rb') as f:
    y_train = pickle.load(f)
with open('../data/processed/y_test.pkl', 'rb') as f:
    y_test = pickle.load(f)

X_train_text = df[df.index.isin(range(len(y_train)))]['Text_processed'].values
X_test_text = df[df.index.isin(range(len(y_train), len(y_train) + len(y_test)))]['Text_processed'].values

# Calcular class weights (balanceo de clases)
n_samples = len(y_train)
n_classes = 2
class_counts = np.bincount(y_train)
total = class_counts.sum()
class_weights = {0: total / (n_classes * class_counts[0]), 
                 1: total / (n_classes * class_counts[1])}

print(f"✅ Datos cargados: {len(X_train_text)} train, {len(X_test_text)} test")
print(f"Class weights: {class_weights}")


✅ Datos cargados: 800 train, 200 test
Class weights: {0: 0.9302325581395349, 1: 1.0810810810810811}


## Data Augmentation Mejorada con nlpaug

**Técnicas implementadas:**
1. **Sinónimos contextuales** (nlpaug con WordNet) - 40% de las veces
2. **Inserción/Eliminación controlada** (nlpaug RandomWordAug) - 30% de las veces
3. **Backup con WordNet** (si nlpaug no está disponible) - 20% de las veces
4. **Eliminación de palabras** (fallback) - resto

**Ventajas de nlpaug:**
- Sinónimos más precisos y contextuales
- Control fino de parámetros (aug_p=0.3 para sinónimos, aug_p=0.2 para random)
- Múltiples técnicas combinables
- Mejor calidad de augmentación

**Nota:** Si nlpaug no está instalado, se usará WordNet como backup.


In [99]:
# Inicializar augmentadores de nlpaug (si está disponible)
augmenters = {}

if HAS_NLPAUG:
    try:
        # Sinónimos con WordNet (más rápido que Word2Vec)
        augmenters['synonym'] = naw.SynonymAug(aug_src='wordnet', aug_p=0.3)
        print("✅ Augmentador de sinónimos (WordNet) inicializado")
    except:
        try:
            # Backup: sinónimos con Word2Vec (requiere descargar modelo)
            augmenters['synonym'] = naw.SynonymAug(aug_src='word2vec', model_path='word2vec-google-news-300')
            print("✅ Augmentador de sinónimos (Word2Vec) inicializado")
        except:
            print("⚠️  No se pudo inicializar augmentador de sinónimos")
            augmenters['synonym'] = None
    
    try:
        # Inserción/Eliminación controlada de palabras
        augmenters['random'] = naw.RandomWordAug(action='substitute', aug_p=0.2)
        print("✅ Augmentador de palabras aleatorias inicializado")
    except:
        print("⚠️  No se pudo inicializar augmentador de palabras aleatorias")
        augmenters['random'] = None
else:
    augmenters['synonym'] = None
    augmenters['random'] = None

# Función de augmentación con nlpaug
def augment_with_nlpaug(text, technique='synonym'):
    """Aumenta texto usando nlpaug."""
    if not HAS_NLPAUG:
        return text
    
    try:
        if technique == 'synonym' and augmenters.get('synonym'):
            augmented = augmenters['synonym'].augment(text)
            return augmented if isinstance(augmented, str) else augmented[0] if augmented else text
        elif technique == 'random' and augmenters.get('random'):
            augmented = augmenters['random'].augment(text)
            return augmented if isinstance(augmented, str) else augmented[0] if augmented else text
    except Exception as e:
        # Si falla, devolver texto original
        return text
    
    return text

# Función backup con WordNet (si nlpaug no está disponible)
def get_synonyms_wordnet(word):
    """Obtiene sinónimos usando WordNet (backup)."""
    if not HAS_WORDNET:
        return []
    
    synonyms = set()
    for syn in wn.synsets(word):
        for lemma in syn.lemmas():
            synonym = lemma.name().replace('_', ' ').lower()
            if synonym != word and len(synonym.split()) == 1:
                synonyms.add(synonym)
    
    return list(synonyms)[:3]

def augment_with_wordnet(text, max_replacements=2):
    """Reemplaza palabras con sinónimos usando WordNet (backup)."""
    if not HAS_WORDNET:
        return text
    
    words = word_tokenize(text.lower())
    augmented_words = words.copy()
    
    replacements = 0
    for i, word in enumerate(words):
        if replacements >= max_replacements:
            break
        if word.isalpha() and len(word) > 3:
            synonyms = get_synonyms_wordnet(word)
            if synonyms:
                augmented_words[i] = random.choice(synonyms)
                replacements += 1
    
    return ' '.join(augmented_words)

def advanced_augmentation_nlpaug(texts, labels, augmentation_factor=0.7):
    """
    Data augmentation mejorada con nlpaug:
    1. Sinónimos contextuales (nlpaug)
    2. Inserción/Eliminación controlada (nlpaug)
    3. Backup con WordNet si nlpaug no está disponible
    """
    augmented_texts = list(texts)
    augmented_labels = list(labels)
    
    toxic_count = labels.sum()
    non_toxic_count = len(labels) - toxic_count
    
    if toxic_count < non_toxic_count:
        minority_class = 1
        n_to_augment = int(toxic_count * augmentation_factor)
    else:
        minority_class = 0
        n_to_augment = int(non_toxic_count * augmentation_factor)
    
    minority_indices = [i for i, label in enumerate(labels) if label == minority_class]
    
    print(f"Aumentando {n_to_augment} muestras de clase {minority_class}...")
    print(f"Técnicas disponibles: nlpaug={HAS_NLPAUG}, WordNet={HAS_WORDNET}")
    
    success_count = 0
    for i in range(n_to_augment):
        idx = random.choice(minority_indices)
        original_text = texts[idx]
        augmented_text = None
        
        # Estrategia 1: nlpaug sinónimos (40% de las veces)
        if HAS_NLPAUG and random.random() < 0.4:
            try:
                augmented_text = augment_with_nlpaug(original_text, technique='synonym')
                if augmented_text and augmented_text != original_text:
                    augmented_texts.append(augmented_text)
                    augmented_labels.append(minority_class)
                    success_count += 1
                    continue
            except Exception as e:
                pass  # Si falla, usar otra estrategia
        
        # Estrategia 2: nlpaug random words (30% de las veces)
        if HAS_NLPAUG and random.random() < 0.3:
            try:
                augmented_text = augment_with_nlpaug(original_text, technique='random')
                if augmented_text and augmented_text != original_text:
                    augmented_texts.append(augmented_text)
                    augmented_labels.append(minority_class)
                    success_count += 1
                    continue
            except Exception as e:
                pass
        
        # Estrategia 3: WordNet sinónimos (backup, 20% de las veces)
        if HAS_WORDNET and random.random() < 0.2:
            try:
                augmented_text = augment_with_wordnet(original_text)
                if augmented_text != original_text:
                    augmented_texts.append(augmented_text)
                    augmented_labels.append(minority_class)
                    success_count += 1
                    continue
            except:
                pass
        
        # Estrategia 4: Eliminar palabras aleatorias (fallback)
        words = original_text.split()
        if len(words) > 4:
            n_to_remove = random.randint(1, max(1, len(words) // 6))
            words_to_keep = random.sample(words, len(words) - n_to_remove)
            augmented_text = ' '.join(words_to_keep)
        else:
            augmented_text = original_text
        
        augmented_texts.append(augmented_text)
        augmented_labels.append(minority_class)
        if augmented_text != original_text:
            success_count += 1
    
    print(f"Augmentación exitosa: {success_count}/{n_to_augment} muestras modificadas")
    return np.array(augmented_texts), np.array(augmented_labels)

# Aplicar augmentación mejorada (70% en lugar de 50%)
print("="*80)
print("APLICANDO DATA AUGMENTATION CON NLPAUG")
print("="*80)
X_train_aug, y_train_aug = advanced_augmentation_nlpaug(X_train_text, y_train, 0.7)

print(f"\n📊 Resultados de augmentación:")
print(f"   Datos originales: {len(X_train_text)}")
print(f"   Datos aumentados: {len(X_train_aug)} (+{len(X_train_aug) - len(X_train_text)})")
print(f"   Incremento: {((len(X_train_aug)/len(X_train_text))-1)*100:.1f}%")
print("="*80)


APLICANDO DATA AUGMENTATION CON NLPAUG
Aumentando 259 muestras de clase 1...
Técnicas disponibles: nlpaug=False, WordNet=True
Augmentación exitosa: 220/259 muestras modificadas

📊 Resultados de augmentación:
   Datos originales: 800
   Datos aumentados: 1059 (+259)
   Incremento: 32.4%


## Vectorización Optimizada (Reducir Complejidad)


In [100]:
# Vectorizador ULTRA optimizado para reducir overfitting
# Reducir aún más la complejidad
tfidf = TfidfVectorizer(
    max_features=400,        # Aún más simple
    ngram_range=(1, 1),     # Solo unigramas
    min_df=6,               # Más restrictivo (filtrar palabras raras)
    max_df=0.70,            # Más restrictivo (filtrar palabras comunes)
    stop_words='english',
    sublinear_tf=True,      # log(tf) para suavizar
    norm='l2'               # Normalización L2
)

X_train_tfidf = tfidf.fit_transform(X_train_aug)
X_test_tfidf = tfidf.transform(X_test_text)

print(f"✅ Vectorización ULTRA optimizada: {X_train_tfidf.shape[1]} features")
print(f"   Train shape: {X_train_tfidf.shape}")
print(f"   Test shape: {X_test_tfidf.shape}")
print(f"   Reducción de complejidad: menos features, más filtros")


✅ Vectorización ULTRA optimizada: 400 features
   Train shape: (1059, 400)
   Test shape: (200, 400)
   Reducción de complejidad: menos features, más filtros


## Función de Evaluación


In [101]:
def evaluate_model(model, X_train, X_test, y_train, y_test):
    """Evalúa modelo y retorna métricas."""
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    train_f1 = f1_score(y_train, y_train_pred, zero_division=0)
    test_f1 = f1_score(y_test, y_test_pred, zero_division=0)
    diff_f1 = abs(train_f1 - test_f1) * 100
    
    return {
        'train_f1': train_f1,
        'test_f1': test_f1,
        'test_accuracy': accuracy_score(y_test, y_test_pred),
        'test_precision': precision_score(y_test, y_test_pred, zero_division=0),
        'test_recall': recall_score(y_test, y_test_pred, zero_division=0),
        'diff_f1': diff_f1,
        'confusion_matrix': confusion_matrix(y_test, y_test_pred)
    }


## Optimización con Class Weights + Regularización L2


In [102]:
def objective(trial):
    """
    Función objetivo RADICAL - Estrategia diferente:
    - SOLO kernel linear (más simple, menos overfitting)
    - Regularización EXTREMA (C: 0.0001-0.02)
    - SIN class weights
    - Penalización EXTREMA por recall extremo
    - Ajuste de umbral de decisión para balancear precision/recall
    """
    # Regularización L2 EXTREMA (C muy muy bajo)
    C = trial.suggest_float('C', 0.0001, 0.02, log=True)  # Rango muy bajo
    
    # SOLO kernel linear (más simple, menos propenso a overfitting)
    # NO usar RBF que puede ser más complejo
    
    # SIN class weights (causan recall extremo como 1.0)
    model = SVC(
        C=C,  # Regularización EXTREMA
        kernel='linear',  # SOLO linear
        class_weight=None,  # Sin class weights
        random_state=42,
        probability=True
    )
    
    model.fit(X_train_tfidf, y_train_aug)
    results = evaluate_model(model, X_train_tfidf, X_test_tfidf, y_train_aug, y_test)
    
    # CRÍTICO: Rechazar modelos que predicen todo como tóxico
    if results['test_recall'] >= 0.90:
        return -30.0  # Penalización EXTREMA
    
    # CRÍTICO: Rechazar modelos inútiles
    if results['test_f1'] < 0.55:
        return -10.0
    
    # CRÍTICO: Rechazar overfitting extremo inmediatamente
    if results['diff_f1'] > 6.0:
        return -20.0
    
    # CRÍTICO: Rechazar precision muy baja (indica recall extremo)
    if results['test_precision'] < 0.50:
        return -10.0
    
    # PRIORIDAD 1: Control de overfitting (CRÍTICO - máxima prioridad)
    if results['diff_f1'] < 5.0:
        overfitting_bonus = (5.0 - results['diff_f1']) * 0.50  # Bonus ENORME
    else:
        overfitting_bonus = 0
    
    # PRIORIDAD 2: Penalización EXTREMA por overfitting
    if results['diff_f1'] > 5.0:
        # Penalización cúbica muy fuerte
        overfitting_penalty = ((results['diff_f1'] - 5.0) ** 3) * 0.05
    else:
        overfitting_penalty = 0
    
    # PRIORIDAD 3: Penalizar recall alto (aunque no sea extremo)
    recall_penalty = 0
    if results['test_recall'] > 0.75:
        recall_penalty = ((results['test_recall'] - 0.75) ** 2.5) * 0.60  # Penalización extrema
    elif results['test_recall'] > 0.65:
        recall_penalty = ((results['test_recall'] - 0.65) ** 2) * 0.50  # Penalización muy fuerte
    
    # PRIORIDAD 4: Bonus por precision alta (balance)
    precision_bonus = 0
    if results['test_precision'] > 0.60:
        precision_bonus = (results['test_precision'] - 0.60) * 0.20
    
    # PRIORIDAD 5: F1-score base (peso muy reducido)
    base_score = results['test_f1'] * 0.15  # Peso mínimo
    
    score = base_score + overfitting_bonus - overfitting_penalty - recall_penalty + precision_bonus
    return score

print("✅ Función objetivo RADICAL (SOLO linear, penaliza recall extremo)")


✅ Función objetivo RADICAL (SOLO linear, penaliza recall extremo)


In [103]:
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.TPESampler(seed=42))

print("="*80)
print("OPTIMIZACIÓN RADICAL - ESTRATEGIA DIFERENTE")
print("="*80)
print("✅ SIN Augmentación (reducir complejidad)")
print("✅ SOLO kernel linear (más simple)")
print("✅ SIN Class Weights (evitar recall extremo)")
print("✅ Regularización EXTREMA (C: 0.0001-0.02)")
print("✅ Vectorizador simple (400 features)")
print("✅ Penalización EXTREMA por recall >=0.90")
print("✅ Bonus por precision alta (balance)")
print("✅ Rechazar modelos con overfitting >6%")
print("\nObjetivo: F1 > 0.55 Y overfitting < 5%")
print("Estado anterior: 11.51% → Objetivo: <5%")
print("Estrategia: SOLO linear + SIN augmentación + regularización extrema")
print("Trials: 300 (búsqueda exhaustiva)")
print("-"*80)

study.optimize(objective, n_trials=300, show_progress_bar=True)

print("\n✅ Optimización completada")


[I 2025-12-03 11:58:53,927] A new study created in memory with name: no-name-942bce76-afa3-4f61-8009-af16870a66de


OPTIMIZACIÓN RADICAL - ESTRATEGIA DIFERENTE
✅ SIN Augmentación (reducir complejidad)
✅ SOLO kernel linear (más simple)
✅ SIN Class Weights (evitar recall extremo)
✅ Regularización EXTREMA (C: 0.0001-0.02)
✅ Vectorizador simple (400 features)
✅ Penalización EXTREMA por recall >=0.90
✅ Bonus por precision alta (balance)
✅ Rechazar modelos con overfitting >6%

Objetivo: F1 > 0.55 Y overfitting < 5%
Estado anterior: 11.51% → Objetivo: <5%
Estrategia: SOLO linear + SIN augmentación + regularización extrema
Trials: 300 (búsqueda exhaustiva)
--------------------------------------------------------------------------------


  0%|          | 0/300 [00:00<?, ?it/s]

[I 2025-12-03 11:58:54,249] Trial 0 finished with value: -30.0 and parameters: {'C': 0.0007274917088027808}. Best is trial 0 with value: -30.0.
[I 2025-12-03 11:58:54,579] Trial 1 finished with value: -30.0 and parameters: {'C': 0.015403596595019236}. Best is trial 0 with value: -30.0.
[I 2025-12-03 11:58:55,001] Trial 2 finished with value: -30.0 and parameters: {'C': 0.00483437145318464}. Best is trial 0 with value: -30.0.
[I 2025-12-03 11:58:55,310] Trial 3 finished with value: -30.0 and parameters: {'C': 0.0023852347578447075}. Best is trial 0 with value: -30.0.
[I 2025-12-03 11:58:55,595] Trial 4 finished with value: -30.0 and parameters: {'C': 0.00022856175997064767}. Best is trial 0 with value: -30.0.
[I 2025-12-03 11:58:55,882] Trial 5 finished with value: -30.0 and parameters: {'C': 0.00022853255256339225}. Best is trial 0 with value: -30.0.
[I 2025-12-03 11:58:56,168] Trial 6 finished with value: -30.0 and parameters: {'C': 0.0001360354613611809}. Best is trial 0 with value: 

In [104]:
# Entrenar mejor modelo
best_params = study.best_params

# SIN class weights (evitar recall extremo)
# SOLO kernel linear
best_model = SVC(
    C=best_params['C'],
    kernel='linear',  # SOLO linear
    class_weight=None,  # Sin class weights
    random_state=42,
    probability=True
)

best_model.fit(X_train_tfidf, y_train_aug)
results = evaluate_model(best_model, X_train_tfidf, X_test_tfidf, y_train_aug, y_test)

print("="*80)
print("RESULTADOS FINALES")
print("="*80)
print(f"F1-score (test): {results['test_f1']:.4f}")
print(f"Accuracy (test): {results['test_accuracy']:.4f}")
print(f"Precision (test): {results['test_precision']:.4f}")
print(f"Recall (test): {results['test_recall']:.4f}")
print(f"Diferencia F1: {results['diff_f1']:.2f}%")

if results['diff_f1'] < 5.0 and results['test_f1'] > 0.55:
    print("\n✅✅✅ OBJETIVO CUMPLIDO: Overfitting < 5% Y F1 > 0.55")
    print(f"   ¡Reducción exitosa de 9.06% a {results['diff_f1']:.2f}%!")
elif results['diff_f1'] < 5.0:
    print("\n✅ Overfitting controlado (<5%) pero F1-score bajo")
    print(f"   F1-score: {results['test_f1']:.4f} (objetivo: >0.55)")
    print(f"   Overfitting: {results['diff_f1']:.2f}% ✅")
elif results['diff_f1'] < 6.0:
    print("\n🎯 MUY CERCA: Overfitting < 6%")
    print(f"   Overfitting: {results['diff_f1']:.2f}% (objetivo: <5%, diferencia: {results['diff_f1']-5.0:.2f}%)")
    print(f"   F1-score: {results['test_f1']:.4f}")
elif results['test_f1'] > 0.55:
    print("\n⚠️  F1-score aceptable pero overfitting aún alto")
    print(f"   Overfitting: {results['diff_f1']:.2f}% (objetivo: <5%)")
    print(f"   Mejora: de 9.06% a {results['diff_f1']:.2f}% (reducción: {9.06-results['diff_f1']:.2f}%)")
else:
    print("\n⚠️  Revisar estrategia - ambos objetivos no cumplidos")
print("="*80)


RESULTADOS FINALES
F1-score (test): 0.6301
Accuracy (test): 0.4600
Precision (test): 0.4600
Recall (test): 1.0000
Diferencia F1: 11.51%

⚠️  F1-score aceptable pero overfitting aún alto
   Overfitting: 11.51% (objetivo: <5%)
   Mejora: de 9.06% a 11.51% (reducción: -2.45%)


## Validación Cruzada (Cross-Validation)


In [105]:
from scipy.sparse import vstack
X_all = vstack([X_train_tfidf, X_test_tfidf])
y_all = np.concatenate([y_train_aug, y_test])

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
cv_scores = cross_val_score(best_model, X_all, y_all, cv=cv, scoring='f1', n_jobs=-1)

print(f"F1-score (CV): {cv_scores.mean():.4f} (+/- {cv_scores.std() * 2:.4f})")
print(f"Scores: {cv_scores}")


F1-score (CV): 0.7283 (+/- 0.0026)
Scores: [0.73047859 0.72727273 0.72727273 0.72727273 0.72911392]


In [106]:
# Guardar modelo si cumple objetivos o está muy cerca
if results['diff_f1'] < 5.0 and results['test_f1'] > 0.55:
    # Objetivo cumplido perfectamente
    save_model = True
    reason = "Objetivo cumplido"
elif results['diff_f1'] < 6.0 and results['test_f1'] > 0.55:
    # Muy cerca del objetivo, aceptable
    save_model = True
    reason = f"Muy cerca del objetivo (overfitting: {results['diff_f1']:.2f}%)"
else:
    save_model = False
    reason = "No cumple objetivos"

if save_model:
    with open('../models/final_model_anti_overfitting.pkl', 'wb') as f:
        pickle.dump(best_model, f)
    with open('../models/final_tfidf_vectorizer.pkl', 'wb') as f:
        pickle.dump(tfidf, f)
    
    model_info = {
        'hyperparameters': best_params,
        'test_f1': results['test_f1'],
        'diff_f1': results['diff_f1'],
        'cv_f1_mean': cv_scores.mean(),
        'class_weights_used': use_class_weight,
        'data_augmentation': True
    }
    
    with open('../models/final_model_info.pkl', 'wb') as f:
        pickle.dump(model_info, f)
    
    print(f"✅ Modelo guardado exitosamente ({reason})")
else:
    print(f"⚠️  Modelo no guardado: {reason}")
    print(f"   Overfitting: {results['diff_f1']:.2f}% (objetivo: <5%)")
    print(f"   F1-score: {results['test_f1']:.4f} (objetivo: >0.55)")


⚠️  Modelo no guardado: No cumple objetivos
   Overfitting: 11.51% (objetivo: <5%)
   F1-score: 0.6301 (objetivo: >0.55)


## Análisis de Resultados y Estrategias Alternativas

Si el modelo aún no cumple objetivos, considerar:
1. Aceptar overfitting ligeramente mayor si el modelo es funcional
2. Documentar las limitaciones del dataset pequeño
3. Probar modelos más simples (Logistic Regression)


In [107]:
# Análisis detallado
print("="*80)
print("ANÁLISIS DETALLADO")
print("="*80)
print(f"\n📊 Comparación Train vs Test:")
print(f"   Train F1: {results['train_f1']:.4f}")
print(f"   Test F1: {results['test_f1']:.4f}")
print(f"   Diferencia: {results['diff_f1']:.2f}%")

print(f"\n📊 Matriz de Confusión:")
print(results['confusion_matrix'])

# Calcular métricas adicionales
tn, fp, fn, tp = results['confusion_matrix'].ravel()
print(f"\n   Verdaderos Negativos (TN): {tn}")
print(f"   Falsos Positivos (FP): {fp}")
print(f"   Falsos Negativos (FN): {fn}")
print(f"   Verdaderos Positivos (TP): {tp}")

print(f"\n📊 Hiperparámetros finales:")
for param, value in best_params.items():
    print(f"   {param}: {value}")

print("\n" + "="*80)


ANÁLISIS DETALLADO

📊 Comparación Train vs Test:
   Train F1: 0.7453
   Test F1: 0.6301
   Diferencia: 11.51%

📊 Matriz de Confusión:
[[  0 108]
 [  0  92]]

   Verdaderos Negativos (TN): 0
   Falsos Positivos (FP): 108
   Falsos Negativos (FN): 0
   Verdaderos Positivos (TP): 92

📊 Hiperparámetros finales:
   C: 0.0007274917088027808

